In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Select just monthly data and just postal sector to make it easier to work with
sql = "SELECT * FROM `ons-fintrans-data-prod.fintrans_reference.census_2024_P003_population_pcds`  "
census = bq.read_bq_table_sql(client, sql)
sql = "SELECT distinct postal_area, postal_district, postal_sector FROM `ons-fintrans-analysis-prod.fin_wip_notebook.nspl_postal_level_lookup`"
nspl = bq.read_bq_table_sql(client, sql)
census_nspl = census.merge(nspl, how = 'left', left_on = 'Postcode_Sectors', right_on = 'postal_sector')
census_nspl

In [ ]:
census_summary = census_nspl.groupby('postal_area').sum('Count').reset_index()
census_summary['postal_area'] = census_summary['postal_area'].replace(" ", "")
#census_summary.to_csv('census_test.csv')

In [ ]:
# Select just monthly data and just postal sector to make it easier to work with
sql = "SELECT cardholder_location, cardholders, time_period_value FROM `ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns`  where time_period = 'Quarter' and merchant_location = 'All' and cardholder_location_level = 'All' and mcg = 'All'"
df_all_q = bq.read_bq_table_sql(client, sql)


In [ ]:
# Select just monthly data and just postal sector to make it easier to work with
sql = "SELECT cardholder_location, cardholders, time_period_value FROM `ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns`  where time_period = 'Quarter' and merchant_location = 'All' and cardholder_location_level = 'POSTAL_AREA' and mcg = 'All'"
df = bq.read_bq_table_sql(client, sql)
df['cardholder_location'] = df['cardholder_location'].replace(" ", "")
df.sort_values('time_period_value')

In [ ]:
df_full = df.merge(census_summary, how = 'outer', left_on = 'cardholder_location', right_on = 'postal_area').sort_values('cardholder_location')
df_full = df_full.loc[(df_full['postal_area'] != 'TD') & (df_full['postal_area'] != 'DG')]

In [ ]:
df_full['year']=df_full['time_period_value'].str[:4]
df_full_2021 = df_full.loc[df_full['year']=='2021']
df_av_2021 = df_full_2021.groupby(['cardholder_location']).mean(['Count','cardholders']).reset_index()
df_av_2021['diff'] = df_av_2021['cardholders']/df_av_2021['Count']
df_av_2021.sort_values('diff').to_csv('test.csv')

In [ ]:
storage_client = storage.Client()
bq.boundary_file_download(storage_client , 'postcode_area','')